# 계산 통계
## 1. 분포

>- 통계에서 분포는 어떤 값과 그 값이 나타날 확률의 집합
>- 확률 질량 함수(probability mass function)으로 분포를 수학적으로 표현 가능
>- thinkbayes 모듈은 다음 [링크](http://thinkbayes.com/thinkbayes.py)에서 다운 가능

In [1]:
from thinkbayes import Pmf

In [2]:
pmf = Pmf()
for x in range(1,7):
    pmf.Set(x, 1/6)

- Pmf는 아무 값도 부여되지 않은 텅 빈 Pmf 객체를 생성
- Set 메서드는 각 값에 확률 값을 설정 (위의 예제에서는 주사위 확률 부여)

In [ ]:
pmf = Pmf()
for word in word_list:
    pmf.Incr(word, 1)

- Incr 메서드는 word_list의 word마다 '확률'을 1씩 높힘 (정규화 되지 않은 확률)

In [ ]:
pmf.Normalize()

- 위 방식으로 모든 빈도 수집 시 Normalize 메서드로 확률 계산

In [ ]:
print(pmf.Prob('the'))

- Prob 메서드로 특정 값의 빈도 출력 가능
___

## 2. 쿠키문제

- 앞서 보았던 쿠키 문제를 Pmf로 해석

In [33]:
pmf = Pmf()

pmf.Set('Bowl 1', 0.5)
pmf.Set('Bowl 2', 0.5)
pmf.GetDict()

{'Bowl 1': 0.5, 'Bowl 2': 0.5}

- Set으로 사전 분포를 설정

In [34]:
pmf.Prob('Bowl 1')

0.5

In [35]:
pmf.Mult('Bowl 1', 0.75)
pmf.Mult('Bowl 2', 0.5)
pmf.GetDict()

{'Bowl 1': 0.375, 'Bowl 2': 0.25}

- 그릇 1에서 바닐라 쿠키를 선택할 우도(확률)은 3/4, 그릇 2에서 바닐라쿠키를 선택할 우도(확률)은 1/2임
- 위 정보를 바탕으로 Mult 메서드로 분포 갱신
- 단, 위와 같이 갱신 시 위 가설은 상호 배반적이며, 전체 포괄적임 -> __정규화 필요__

In [6]:
pmf.Normalize()
pmf.Prob('Bowl 1')

0.6000000000000001

- 각 가설에 대한 사후 확률을 포함하는 사후 분포로 나타남
- 그릇 1의 사후확률은 0.6임

In [7]:
pmf.GetDict()

{'Bowl 1': 0.6000000000000001, 'Bowl 2': 0.4}

___
## 3. 베이지안 프레임워크
- 앞 장에서 다룬 코드를 클래스로 정의

In [8]:
class Cookie(Pmf):
    
    def __init__(self, hypos):
        Pmf.__init__(self)
        for hypo in hypos:
            self.Set(hypo, 1)
        self.Normalize()
        
    
    def Update(self, data):
        for hypo in self.Values():
            like = self.Likelihood(data, hypo)
            self.Mult(hypo, like)
        self. Normalize()
    
    
    mixes = {
    'Bowl 1': dict(vanilla = 0.75, chocolate = 0.25),
    'Bowl 2': dict(vanilla = 0.5, chocolate = 0.5)
    }
    
    
    def Likelihood(self, data, hypo):
        mix = self.mixes[hypo]
        like = mix[data]
        return like

-	\__init__ 메서드는 각 가설별로 동일한 사전확률을 부여
-	Update 메서드는 우도(likelihood)를 사전확률에 곱하고, 정규화 하여 사후 확률을 도출
-	Likelihood 메서드는 Update에서 사용할 우도를(mixes) 전달
- mixes객체는 "각 가설 별로 계산된 우도"를 담고 있음

In [9]:
hypos = ['Bowl 1', 'Bowl 2']
pmf = Cookie(hypos)
pmf.GetDict()

{'Bowl 1': 0.5, 'Bowl 2': 0.5}

- 'Bowl 1'과 'Bowl 2' 2종의 가설 설정
- Cookie 클래스로 'Bowl 1'과 'Bowl 2'의 사전확률 설정 (동일한 사전확률)

In [10]:
pmf.Update('vanilla')
pmf.GetDict()

{'Bowl 1': 0.6000000000000001, 'Bowl 2': 0.4}

- Update 메서드로 바닐라에 대한 사후 확률 계산
- Update 안의 Mult 메서드로 분포 갱신 $(0.5 * 0.75, 0.5*0.5)$
- 위의 분포를 Normalize 메서드로 정규화 $(\frac{0.5 * 0.75}{\sum}, \frac{0.5*0.5}{\sum})$

In [11]:
dataset = ['vanilla', 'chocolate', 'vanilla']
for data in dataset:
    pmf.Update(data)
pmf.GetDict()

{'Bowl 1': 0.627906976744186, 'Bowl 2': 0.37209302325581395}

- for문을 사용하여 한개 이상의 쿠키를 집는 경우를 일반화 하는 것이 가능
___

## 4. 몬티 홀 문제
- 마찬가지로 앞 장에서 다루었던 문제를 클래스로 구현 *(몬티홀 참가자는 A를 선택한 상황)*

In [12]:
class Monty(Pmf):
    
    def __init__(self, hypos):
        Pmf.__init__(self)
        for hypo in hypos:
            self.Set(hypo, 1)
        self.Normalize()

        
    def Update(self, data):
        for hypo in self.Values():
            like = self.Likelihood(data, hypo)
            self.Mult(hypo, like)
        self. Normalize()
    
    
    def Likelihood(self, data, hypo):
        if hypo == data:
            return 0
        elif hypo == 'A':
            return 0.5
        else:
            return 1

- 앞서 쿠키 사례와 \__init__, Update 메서드는 동일
- Likelihood 메서드는 data(몬티 홀이 연 문)일 경우, 차가 선택한 'A' 뒤에 있을 경우, 'C'에 있을 경우의 우도를 담고 있음

In [13]:
hypos = list('ABC')
pmf = Monty(hypos)
pmf.GetDict()

{'A': 0.3333333333333333, 'B': 0.3333333333333333, 'C': 0.3333333333333333}

- 사전 확률은 쿠키 클래스 때와 마찬가지로 동일

In [14]:
data = 'B'
pmf.Update(data)
pmf.GetDict()

{'A': 0.3333333333333333, 'B': 0.0, 'C': 0.6666666666666666}

- 우도를 반영한 몬티홀 결과 값은, 'A'를 선택하고 'B'문이 열렸을 때 'C'문에 차가 있을 확률이 2배 높다고 볼 수 있음
___

## 5. 프레임워크 캡슐화

- 베이지안 프레임워크에서 동일한 요소들이 있으며 캡슐화가 가능
- 동일한 요소들은 \__init__, Update, (print) 기능으로 정의
- 우도를 담은 Likelihood 메서드만 프레임워크 별로 정의

In [ ]:
class Suite(Pmf):
    """Represents a suite of hypotheses and their probabilities."""

    def Update(self, data):
        """Updates each hypothesis based on the data.
        data: any representation of the data
        returns: the normalizing constant
        """
        for hypo in self.Values():
            like = self.Likelihood(data, hypo)
            self.Mult(hypo, like)
        return self.Normalize()


    def Print(self):
        """Prints the hypotheses and their probabilities."""
        for hypo, prob in sorted(self.Items()):
            print(hypo, prob)

            
#### 외 기타 기능들은 think bayes에 포함
#### __init__은 Suite 클래스에서 호출한 Pmf 클래스에서 작동

In [15]:
from thinkbayes import Suite

class Monty(Suite):
    def Likelihood(self, data, hypo):
        if hypo ==data:
            return 0
        elif hypo == 'A':
            return 0.5
        else:
            return 1

- Monty 클래스는 thinkbayes의 Suite 클래스를 상속받음
- Likelihood만 프레임워크에 맞춰 추가하여 Monty 프레임워크 구축 *('A' 선택)*

In [16]:
suite = Monty('ABC')
suite.Update('B')
suite.Print()

A 0.3333333333333333
B 0.0
C 0.6666666666666666


___
## 6. M&M 문제
- 색구성이 다른 M&M이 두봉지 있음 M&M(1994), M&M(1996)
- 각 봉지에서 M&M을 꺼냈을 때 한알은 노란색, 한알은 녹색임
- 노란색이 M&M(1994)일 확률은?

In [17]:
mix94 = dict(brown=30, yellow=20, red=20, green=10, orange=10, tan=10)
mix96 = dict(blue=24, green=20, orange=16, yellow=14, red=13, brown=13)
mix96

{'blue': 24, 'brown': 13, 'green': 20, 'orange': 16, 'red': 13, 'yellow': 14}

- 각 봉지 별 색구성 정의

In [23]:
hypo1 = dict(bag1 = mix94, bag2 = mix96)
hypo2 = dict(bag1 = mix96, bag2 = mix94)

hypotheses = dict(A=hypo1, B = hypo2)
hypotheses

{'A': {'bag1': {'brown': 30,
   'green': 10,
   'orange': 10,
   'red': 20,
   'tan': 10,
   'yellow': 20},
  'bag2': {'blue': 24,
   'brown': 13,
   'green': 20,
   'orange': 16,
   'red': 13,
   'yellow': 14}},
 'B': {'bag1': {'blue': 24,
   'brown': 13,
   'green': 20,
   'orange': 16,
   'red': 13,
   'yellow': 14},
  'bag2': {'brown': 30,
   'green': 10,
   'orange': 10,
   'red': 20,
   'tan': 10,
   'yellow': 20}}}

- 가설을 코드화: 가설A에서 1번 봉지는 M&M(1994), 2번 봉지는 M&M(1996); 가설B에서는 그 반대
- 가설 별로 확률 분포 딕셔너리에 저장

In [19]:
class M_and_M(Suite):
    
    def Likelihood(self, data, hypo):
        bag, color = data
        mix = hypotheses[hypo][bag]
        like = mix[color]
        return like

- 내용 추가 필요

In [60]:
suite = M_and_M('AB')

suite.Update(('bag1', 'yellow'))
suite.Update(('bag2', 'green'))
suite.Print()

A 0.7407407407407407
B 0.2592592592592592


- 해석 추가 필요

In [87]:
suite = M_and_M('AB')
suite.d

{'A': 0.5, 'B': 0.5}

In [ ]:
suite.Update(('bag1', 'yellow'))

    def Update(self, data):
        """Updates each hypothesis based on the data.
        data: any representation of the data
        returns: the normalizing constant
        """
        for hypo in self.Values():
            like = self.Likelihood(data, hypo)
            self.Mult(hypo, like)
        return self.Normalize()
    
    def Likelihood(self, data, hypo):
        bag, color = data
        mix = hypotheses[hypo][bag]
        like = mix[color]
        return like

In [88]:
hypos = suite.Values()

bag = 'bag1'
color = 'yellow'


mix = hypotheses[hypos[0]][bag]
like = mix[color]
suite.Mult(hypos[0], like)

mix = hypotheses[hypos[1]][bag]
like = mix[color]
suite.Mult(hypos[1], like)

suite.Normalize()


17.0

In [89]:
hypos = suite.Values()

bag = 'bag2'
color = 'green'


mix = hypotheses[hypos[0]][bag]
like = mix[color]
suite.Mult(hypos[0], like)

mix = hypotheses[hypos[1]][bag]
like = mix[color]
suite.Mult(hypos[1], like)
suite.Normalize()

15.882352941176471

In [90]:
suite.d

{'A': 0.7407407407407407, 'B': 0.2592592592592592}

In [85]:
suite.Normalize()

270.0

In [86]:
suite.d

{'A': 0.7407407407407408, 'B': 0.25925925925925924}

In [32]:
hypotheses['A']['bag1']['yellow']

20

In [28]:
(hypotheses['A']['bag1']['yellow']*0.5, hypotheses['A']['bag2']['green']*.5)

(10.0, 10.0)

In [29]:
(hypotheses['B']['bag1']['yellow']*0.5, hypotheses['B']['bag2']['green']*.5)

(7.0, 5.0)

In [ ]:
for hypo in self.Values():
    like = self.Likelihood(data, hypo)
    self.Mult(hypo, like)


In [30]:
hyy = hypotheses

In [39]:
M_and_M("AB").Values()

['A', 'B']

In [43]:

M_and_M("AB").d.get('A',0)

0.5

In [22]:
hypotheses

{'A': {'bag1': {'brown': 30,
   'green': 10,
   'orange': 10,
   'red': 20,
   'tan': 10,
   'yellow': 20},
  'bag2': {'blue': 24,
   'brown': 13,
   'green': 20,
   'orange': 16,
   'red': 13,
   'yellow': 14}},
 'B': {'bag1': {'blue': 24,
   'brown': 13,
   'green': 20,
   'orange': 16,
   'red': 13,
   'yellow': 14},
  'bag2': {'brown': 30,
   'green': 10,
   'orange': 10,
   'red': 20,
   'tan': 10,
   'yellow': 20}}}

In [58]:
suite = M_and_M('AB')

In [59]:
suite.Mult('A', 20)
suite.Mult('B', 20)
suite.d

{'A': 10.0, 'B': 10.0}

In [53]:
suite.Normalize()

10.5

In [54]:
suite.d

{'A': 0.9523809523809523, 'B': 0.047619047619047616}

In [55]:
suite.Mult('A', 20)

In [56]:
suite.Normalize()

19.095238095238095

In [57]:
suite.d

{'A': 0.997506234413965, 'B': 0.0024937655860349127}